
steps:
1.   get the csv with all the messages and the coders' ratings
2.   extract the messages
3.   calculate ground-truth ratings
4.   generate the personas
5.   get the desired set of messages
6.   loop through everything and get the desired outputs
7.   store that
8.   calculate the difference between that and the ground-truth ratings

NEED TO DO after getting coders' ratings:
3, 6, 7

# Main Methods here

In [ ]:
my_api_key = "" # from PASCL's GPT-4 account

In [ ]:
pip install openai

In [ ]:
import openai
import os
from openai import OpenAI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.environ["OPENAI_API_KEY"] = my_api_key
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [ ]:
# extract the messages, sort by reactivity
df_sorted = extract_messages(10, 0) # 10 messages, most effect on well-being

In [ ]:
# calculate ground-truth ratings

In [ ]:
# generate the personas
numbers = [1, 10, 20, 50, 70, 100, 150, 200, 250, 300, 350, 400, 450, 500]
# trying out different numbers of participants to see which number will best reflect the real-world
averages = findBest(numbers)
#print(numbers)
#print(averages)
plot(numbers, averages)

In [ ]:
# say, n = 100
participants, attributes = generate_participants(100)
personas = generatePersonas(participants)

In [ ]:
#calls(messages, personas)
# get the desired outputs and store them
# calculate precision and recall

# Helper Methods here

In [ ]:
# get the messages and sort them by reactivity
def extract_messages(n, type):
  df = pd.read_csv("/content/drive/MyDrive/css summer 2024/stanford/rating tweets/Rating Tweets - Sheet1.csv")
  df = df[df['Remove'] != 1]
  df = df.drop(columns=['Remove'])
  messages = df.iloc[:,3]
  df['reaction'] = abs(df.iloc[:, [4, 5, 6]].mean(axis=1) - 50) # how much of an effect the post has on well-being
  df_sorted = df.sort_values(by='reaction') # 0 --> 50
  if type == 0: # n posts which provoke the strongest reaction
    return df_sorted.iloc[-n:, [3, -1]]
  elif type == 1: # n posts which provoke the weakest reaction
    return df_sorted.iloc[0:n, [3, -1]]
  elif type == 2: # random n posts
    return df_sorted.sample(n).iloc[:, [3, -1]]

In [ ]:
# using my spreadsheet to get the attributes and real-world weights and then generate the participants
def generate_participants(n):
    df = pd.read_csv('/content/drive/MyDrive/css summer 2024/stanford/Fictional Participants - Sheet2.csv')
    attributes = {
        'age_group':
        {'categories': df.iloc[:, 0].dropna().tolist(), 'weights': df.iloc[:, 1].dropna().tolist()},
        'marital_status':
        {'categories': df.iloc[:, 2].dropna().tolist(), 'weights': df.iloc[:, 3].dropna().tolist()},
        'race':
        {'categories': df.iloc[:, 4].dropna().tolist(), 'weights': df.iloc[:, 5].dropna().tolist()},
        'sex':
        {'categories': df.iloc[:, 6].dropna().tolist(), 'weights': df.iloc[:, 7].dropna().tolist()},
        'education':
        {'categories': df.iloc[:, 8].dropna().tolist(), 'weights': df.iloc[:, 9].dropna().tolist()},
        'income':
        {'categories': df.iloc[:, 10].dropna().tolist(), 'weights': df.iloc[:, 11].dropna().tolist()},
        'mental_health':
        {'categories': df.iloc[:, 12].dropna().tolist(), 'weights': df.iloc[:, 13].dropna().tolist()},
        'political_affiliation':
        {'categories': df.iloc[:, 14].dropna().tolist(), 'weights': df.iloc[:, 15].dropna().tolist()}
    }
    data = {}
    for attribute, info in attributes.items():
        data[attribute] = np.random.choice(info['categories'], size=n, p=info['weights'])

    df = pd.DataFrame(data)

    return df, attributes

In [ ]:
# to get the distributions of each attribute and see how closely they match real-world data
def verify_dataset(df, attributes):
    total_samples = len(df)
    verification_results = {}
    total_deviations = []

    for column, info in attributes.items():
        observed_counts = df[column].value_counts().reindex(info['categories'], fill_value=0)
        observed_percentages = observed_counts / total_samples
        original_weights = pd.Series(info['weights'], index=info['categories'])

        deviations = observed_percentages - original_weights
        total_deviations.extend(deviations.abs().tolist())

        verification_results[column] = {
            'observed_percentages': observed_percentages,
            'original_weights': original_weights,
            'deviations': deviations,
            'average_deviation': deviations.abs().mean()
        }

    overallAverageDeviation = np.mean(total_deviations)
    return verification_results, overallAverageDeviation

In [ ]:
# find the number of participants which will most-closely mirror real-world data
def findBest(numbers):
  averages = []
  for num in numbers:
    participants, attributes = generate_participants(num)
    verification_results, overallAverageDeviation = verify_dataset(participants, attributes)
    averages.append(overallAverageDeviation)
  return averages

In [ ]:
# plot two lists
def plot(a, b):
  plt.figure(figsize=(10, 6))
  plt.plot(a, b)
  plt.xlabel('Number of Participants')
  plt.ylabel('Average Deviation')
  plt.title('Average Deviation vs. Number of Participants')
  plt.show()

In [ ]:
# get the system call string given the attributes of a persona
def generatePersonas(df):
  personas = []
  for i in range(len(df)):
    age = df.iloc[i, 0]
    marital_status = df.iloc[i, 1]
    race = df.iloc[i, 2]
    sex = df.iloc[i, 3]
    education = df.iloc[i, 4]
    income = df.iloc[i, 5]
    mental_health = df.iloc[i, 6]
    political_affiliation = df.iloc[i, 7]
    persona = f"You are a {age} year-old, {marital_status}, {race} {sex} with {education} making {income} per year, living in the United States. You have {mental_health} and your political affiliation is {political_affiliation}."
    personas.append(persona)
  return personas

In [ ]:
# just an API call
def api_call(client, system_content, user_content):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
  )
  return response.choices[0].message.content

In [ ]:
# given the set of messages and personas, making the calls
def calls(messages, personas):
  user_content = """
  You are presented with a social media post. On a scale from 0 to 100, please indicate how you would feel,
  as a social media user, after seeing the social media post. Specifically, answer these questions:
  (1) if you would feel satisfied with yourself,
  (2) if you have a positive attitude towards yourself, and
  (3) if you would feel that you have a number of good qualities.
  Please just output three numbers, separated by a comma and a space, and nothing else.
  An example format is: "50, 50, 50" and nothing else.

  SOCIAL MEDIA POST:
  """
  for message in messages:
    for persona in personas:
      user_content += message
      system_content = persona
      response = api_call(client, system_content, user_content)
      print(response)
      print("--------------------------------------------------------------")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def calculate_metrics(ground_truth, predicted):
    accuracy = accuracy_score(ground_truth, predicted)
    precision = precision_score(ground_truth, predicted, average='macro')
    recall = recall_score(ground_truth, predicted, average='macro')
    f1 = f1_score(ground_truth, predicted, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

# Don't need to run this

In [ ]:
# testing to make sure that it works
user_content = "What is the meaning of life?"
system_content = "You are a helpful assistant."

response = api_call(client, system_content, user_content)
print(response)